# Лабораторная работа 3. Pandas, метод ближайших соседей и решающие деревья.

Name:Zhadyra

Student ID: 16BD02151

Email:oralkhanova.zhadyra@gmail.com

In [17]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
from sklearn.metrics import roc_auc_score
import sklearn.ensemble as ensemble
import math
import matplotlib.pyplot as plt

## Часть 1: Pandas

#### Ответьте на вопросы о данных по авиарейсам в США.

Данные: http://stat-computing.org/dataexpo/2009/2008.csv.bz2
(обратите внимание, что распаковывать этот файл не обязательно — функция `pandas.read_csv` умеет читать из архивов автоматически)

Описание: http://stat-computing.org/dataexpo/2009/the-data.html

1. Какая из причин отмены рейса (`CancellationCode`) была самой частой? (расшифровки кодов можно найти в описании данных)
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни?
4. Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится?
5. Найдите для каждого аэропорта среднее время полета (`AirTime`) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?
6. Найдите аэропорт, у которого наибольшая доля задержанных (`DepDelay > 0`) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию `filter` после `groupby`).

In [ ]:
data = pd.read_csv("2008.csv.bz2")
data.describe()

## Часть 2: метрические методы и категориальные признаки

Все дальнейшие эксперименты предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [3]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [4]:
data.shape

(32769, 10)

In [5]:
# доля положительных примеров
data.ACTION.mean()

0.9421099209618847

In [6]:
# число значений у признаков
for col_name in data.columns:
    print (col_name, len(data[col_name].unique()))

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [7]:
from sklearn.cross_validation import train_test_split
X_train=train_test_split(data.iloc[:, 1:], data.iloc[:, 0],test_size=0.3, random_state=241)
X_test=train_test_split(data.iloc[:, 1:], data.iloc[:, 0],test_size=0.3, random_state=241)
y_train=train_test_split(data.iloc[:, 1:], data.iloc[:, 0],test_size=0.3, random_state=241)
y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],test_size=0.3, random_state=241)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре.

Проще всего будет определить метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba). Можно реализовать метод k ближайших соседей и самостоятально — в этом случае учитите, что он должен возвращать оценку вероятности, то есть отношение объектов первого класса среди соседей к числу соседей).

Постарайтесь уделить особое внимание эффективности кода — при реализации метрик "в лоб" вы можете столкнуться с очень большим временем выполнения.

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

#### Какая функция расстояния оказалась лучшей?

In [8]:
x=20
y=10
def indicator_dist(x, y):
    return (x != y).sum()
def smoothy_indicator_dist(x, y):
    return np.array([(x[c] != y[c]) + (x[c] == y[c]) * Hash_sid[c].get(x[c], 0) for c in range(9)]).sum()
def importance_dist(x, y):
    return np.array([(x[c] != y[c]) * Hash_id[c].get(x[c], 0) * Hash_id[c].get(y[c], 0) for c in range(9)]).sum()

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось получить?

Для подбора можно использовать любые средства из sklearn.

In [12]:

def MakeCounters(train, extractable_data, extractable_data_ans):
    train_modified = pd.DataFrame(index=train.index)
    for feature in train.columns:
        vc = extractable_data[feature].value_counts()
        vc_t = (extractable_data[feature] * extractable_data_ans).value_counts()
        train_modified[feature + "_counts"] = pd.Series([vc.get(x, 0) for x in train[feature]], index=train.index),
        train_modified[feature + "_successes"] = pd.Series([vc_t.get(x, 0) for x in train[feature]], index=train.index)
        train_modified[feature + "_cs"] = (train_modified[feature + "_successes"] + 1) / (train_modified[feature + "_counts"] + 2)
        return train_modified

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`successes` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанные по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [28]:
def MakeCounters(train, extractable_data, extractable_data_ans):
    train_modified = pd.DataFrame(index=train.index)
    for feature in train.columns:
        vc = extractable_data[feature].value_counts()
        vc_t = (extractable_data[feature] * extractable_data_ans).value_counts()
        train_modified[feature + "_counts"] = pd.Series([vc.get(x, 0) for x in train[feature]], index=train.index)
        train_modified[feature + "_successes"] = pd.Series([vc_t.get(x, 0) for x in train[feature]], index=train.index),
        train_modified[feature + "_cs"] = (train_modified[feature + "_successes"] + 1) / (train_modified[feature + "_counts"] + 2),
    return train_modified

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $(f_i, f_j)$, $i < j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$ (желательно через какой-нибудь специальный символ во избежание коллизий). Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

In [13]:
def MakePairFeatures(train):
    train_modified = train.copy()
    for i, feature1 in enumerate(train.columns):
        for feature2 in train.columns[i+1:]:
            train_modified[feature1 + "+" + feature2] = train[feature1].astype(str) + "#" + train[feature2].astype(str)
    return train_modified

## Часть 3: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

#### 2. Настройте случайный лес, подобрав такое число деревьев `n_estimators`, при котором ошибка выходит на асимптоту. Какое качество на тестовой выборке он дает?

In [ ]:
n_set = np.arange(1, 20)
results = np.zeros(n_set.shape[0])
overfitting = np.zeros(n_set.shape[0])
for i, n in enumerate(n_set):
    for k in range(5):
        clf = ensemble.RandomForestClassifier(n_estimators = n, max_depth=8, min_samples_leaf=15, n_jobs = -1)
        clf = clf.fit(mod_train_wf, y_train)
        results[i] += sklearn.metrics.roc_auc_score(y_test, clf.predict_proba(mod_test_wf)[:, 1])
        overfitting[i] += sklearn.metrics.roc_auc_score(y_train, clf.predict_proba(mod_train_wf)[:, 1])
        results[i] /= 5
        overfitting[i] /= 5
plt.plot(n_set, results)
plt.plot(n_set, overfitting)
plt.show()

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

In [ ]:
n_set = np.arange(1, 20)
results = np.zeros(n_set.shape[0])
overfitting = np.zeros(n_set.shape[0])
    for i, n in enumerate(n_set):
        for k in range(5):
            clf = ensemble.RandomForestClassifier(n_estimators = n, max_depth=5, min_samples_leaf=20, n_jobs = -1)
            clf = clf.fit(mod_train_wf, y_train)
            results[i] += sklearn.metrics.roc_auc_score(y_test, clf.predict_proba(mod_test_wf)[:, 1])
            overfitting[i] += sklearn.metrics.roc_auc_score(y_train, clf.predict_proba(mod_train_wf)[:, 1])
        results[i] /= 5
        overfitting[i] /= 5    
        plt.plot(n_set, results)
    plt.plot(n_set, overfitting)
    plt.show()

In [ ]:
n_set = np.arange(1, 20)
results = np.zeros(n_set.shape[0])
overfitting = np.zeros(n_set.shape[0])
for i, n in enumerate(n_set):
    for k in range(5):
        clf = ensemble.RandomForestClassifier(n_estimators = n, max_depth=8, min_samples_leaf=15, n_jobs = -1)
        clf = clf.fit(mod_train_wf, y_train)
        results[i] += sklearn.metrics.roc_auc_score(y_test, clf.predict_proba(mod_test_wf)[:, 1])
        overfitting[i] += sklearn.metrics.roc_auc_score(y_train, clf.predict_proba(mod_train_wf)[:, 1])
    results[i] /= 5
    overfitting[i] /= 5
plt.plot(n_set, results)
plt.plot(n_set, overfitting)
plt.show()
print(results.max())

Здесь вы можете поделиться своими мыслями о задании.

In [ ]:
Задание было сложным, думаю надо сначало практиковаться с задачами по 

А здесь вставьте смешную картинку.

In [ ]:
https://www.google.kz/search?q=funny+memes+about+studying&rlz=1C1GGRV_enKZ779KZ779&source=lnms&tbm=isch&sa=X&ved=0ahUKEwjviI6j_o3eAhWKtosKHawvAyYQ_AUIDigB&biw=1242&bih=597#imgrc=sOXAzJUl3Vc5WM:

А здесь посоветуйте преподавателям хороший фильм или сериал.

In [ ]:
Поезд в Пуссан